# Chapter 4. 符号计算
符号计算对人来说不容易，对计算机更是如此。

有不少算法，尤其优化算法，具有分段性，强依赖于值，这些是不可符号计算的。但有不少任务还是可以符号化的，
比如解微分方程，概率分布等。

我们采用 sympy 作为符号计算器。至于专用软件，MATLAB 符号计算十分简陋，不如 mathematica。

## 基础

不同于 mathematica 的专为符号计算设计，sympy 由于 python 语言限制，符号和表达式都要手动定义。

注意 assumption 部分并不智能，它只会做最简单的判断：

> The assumptions system is intended to be efficient though: it is expected many more complex queries
> will not be fully resolved. This is because assumptions queries are primarily used internally by SymPy
> as part of low-level calculations. Making the system more comprehensive would slow SymPy down.

In [1]:
import sympy
# 定义变量
a,b,c=sympy.symbols("a b c") # 定义多变量
a1,a2,a3=sympy.symbols("a1:4") # 批量定义
sympy.symbols('x5(:2)') # (x50, x51)
sympy.symbols('x:2(1:3)') # (x01, x02, x11, x12)
sympy.symbols("varphi \\varphi") # 拉丁字母，latex；注意底层是字符串，显示同一个符号但是算两个变量
# 解析 `S`
1/2 # 0.5
sympy.S(1)/2 # 1/2
sympy.S("1/2") # 1/2
sympy.S('A * x**2 + B * x + C') # A x^2 + B x + C
# 添加假设
sympy.symbols('f,g,h', cls=sympy.Function) # 函数
sympy.symbols('n', positive=True, integer=True, real=True)
# 通过全局假设简化
x,y=sympy.symbols("x y", positive=True)
sympy.simplify(sympy.Abs(x+y)) # x+y
# 临时假设 `Q``
from sympy.assumptions import refine, assuming, Q, ask
x,y=sympy.symbols("x y")
with assuming(Q.integer(x), Q.integer(y)): # 添加假设
    ask(Q.integer(x + y)) # returns `True`` # 检测假设成立
# 当然这个假设不是很智能
with assuming(Q.positive(x-1)): # 添加假设
    ask(Q.positive(x)) # None rather than True
# 通过临时假设简化。假设之间通过 `&` 连接
refine(sympy.Abs(x+y), Q.positive(x) & Q.positive(y)) # x+y
refine(sympy.Abs(x+y), Q.positive(x+y-1)) # x+y

None # avoid display

## 解方程

In [2]:
from sympy import solveset, solve, nsolve
from sympy import symbols, S, Interval
from sympy import pi
from sympy.functions import sin, cos
x,y,z=symbols('x y z')
# 多项式方程
solveset(x**2 - y, x) # [-sqrt(y), sqrt(y)]
solveset(x**4 - 256, x) # [-4, 4, -4*I, 4*I]
xpos=symbols('xpos', real=True)
solveset(xpos**4 - 256, xpos) # [-4, 4]
solveset(x**4 - 256, x, domain=S.Reals) # {−4,4} 设置定义域
# 某些方程
solveset(sin(x), x, Interval(-pi, pi)) # {0,−π,π}
solveset(sin(x), x) # {2nπ∣n∈Z}∪{2nπ+π∣n∈Z}
solveset(cos(x) - x, x) # 超越方程 {x∣x∈C∧−x+cos(x)=0}
# 方程组
solve([x + y - 2*z, y + 4*z], [x, y]) # {x: 6*z, y: -4*z}
solve([x**2 + y - 2*z, y + 4*z], x, y) # [(-sqrt(6)*sqrt(z), -4*z), (sqrt(6)*sqrt(z), -4*z)]
nsolve((x**2 + y - 2, y**3 + x + 4), (x,y), (-1, -1)) # 数值解 [ −1.81588898509945, −1.29745280620551 ]
solve([x**2 + y - 2, y**3 + x + 4], [x,y]) # 无解析解 []
# 单变量微分方程
from sympy import Function, dsolve
y = Function('f')(x)
dsolve(y.diff(x,x) + 9*y, y) # f(x)=C1 sin(3x)+C2 cos(3x)
# 方程组
f, g = y = symbols("f g", cls=Function)
dsolve([
    f(x).diff(x)-g(x), 
    g(x).diff(x)-f(x),
], [f(x), g(x)]) # [Eq(f(x), -C1*exp(-x) + C2*exp(x)), Eq(g(x), C1*exp(-x) + C2*exp(x))]

None

## 矩阵

In [3]:
from sympy import S, simplify
from sympy.matrices import Matrix
# 定义-暴力版
m=Matrix(S("[[5,y,4],[x,1,z]]")) # 暴力解析
n=Matrix(S("[a,b,c]")) # 一维数组默认为列向量
m*n # [ a+by+4c, ax+b+cz ]
# 求逆
simplify(Matrix(S("[[cos(x),sin(x)],[-sin(x),cos(x)]]"))**-1) # [[cos(x),-sin(x)],[sin(x),cos(x)]]

None

## 概率分布
`sympy.stats` 模块实现了概率分布的符号计算。对一个概率变量有以下函数

| 函数 | 意义 |
|-|-|
| `P(condition)` | 概率 |
| `E(expression)` | 期望 |
| `variance(expression)` | 方差 |
| `H(expression)` | 熵 |
| `density(expression)` | 概率密度函数解析式 |
| `sample(expression)` | 产生一个样本 |



In [4]:
from sympy.stats import P, E, H, variance, density, sample, Normal
from sympy import simplify, And
Z = Normal('Z', 1, 2) # 正态分布
Y = Normal('Y', 1, 2)
simplify(P(Z>3)) # 1/2 - erf(sqrt(2)/2)/2
E(Z) # 1
variance(Z) # 4
H(Z) # 1/2 + log(pi)/2 + 3*log(2)/2
# density(Z*Y) # 一坨，很慢
sample(Z) # 有点慢
simplify(P(Z>2))

1/2 - erf(sqrt(2)/4)/2